# Продвинутое машинное обучение: ДЗ №3

В этом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена. Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее. В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:
https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip 

In [1]:
# Используемые бибилиотеки
import numpy as np
import string
import random
from collections import Counter

## Содержание: <a class="anchor" id="zero-bullet"></a>  
* [1. Частотный метод на базе униграмм](#1)  
* [2. Частотный метод на базе биграмм](#2)  
* [3. MCMC-метод](#3)  
* [4. Расшифровка сообщения](#4)  
* [5. Бонус 1](#5)  
* [6. Бонус 2](#6)  

## 1. Частотный метод на базе униграмм <a class="anchor" id="1"></a>  

Условие задачи: *Реализуйте базовый частотный метод по Шерлоку Холмсу:*  
 - подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);  
 - возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;  
 - расшифруйте их таким частотным методом.

In [2]:
# Файл "Анна Каренина" имеет больше символов, поэтому используем его в качестве обучающего
with open('data/AnnaKarenina.txt', 'r', encoding='utf-8') as f:
    train_text = f.read()
print('Фрагмент обучающего текста:')
print(train_text[13:209])

Фрагмент обучающего текста:
«Анна Каренина», один из самых знаменитых романов Льва Толстого, начинается ставшей афоризмом фразой: «Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему». 


In [3]:
with open('data/WarAndPeace.txt', 'r', encoding='utf-8') as f:
    test_text = f.read()

В качестве тестового набора используем фрагмент из тестового файла

In [4]:
START, STOP = 5178, 5496
print('Тестовый текст:')
print(test_text[START:STOP])

Тестовый текст:
Австрия никогда не хотела и не хочет войны. Она предает нас. Россия одна должна быть спасительницей Европы. Наш благодетель знает свое высокое призвание и будет верен ему. Вот одно, во что я верю. Нашему доброму и чудному государю предстоит величайшая роль в мире, и он так добродетелен и хорош, что Бог не оставит его


In [5]:
test_text = test_text[START:STOP]

Создадим необходимые классы и функции для кодирования и декодирования частотным методм

In [6]:
# Функция для шифровки путем случайной перестановки
def encode(text):
    alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
    permutation = ''.join(random.sample(alphabet, len(alphabet)))
    transtable = str.maketrans(alphabet, permutation)
    encoded_text = text.lower().translate(transtable)
    encoded_text = ''.join([char for char in encoded_text if char in permutation])
    return encoded_text    

In [7]:
# Частотный декодер
class Decoder:
    
    def __init__(self, n_gram_len = 1):
        self.n_gram_len = n_gram_len
        self._alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
    
    def fit(self, text):
        text = ''.join([char for char in text.lower() if char in self._alphabet])
        splitted_text = [text[i:i + self.n_gram_len] for i in range(len(text) - self.n_gram_len + 1)]
        self._frequency_dict = Counter(splitted_text)
        sort_indx = np.argsort(list(self._frequency_dict.values()))[::-1]
        self._alphabet_sorted = np.array(list(self._frequency_dict.keys()))[sort_indx]
    
    def decode(self, text):
        original_len = len(text)
        text += ' ' * (original_len % self.n_gram_len)
        splitted_text = [text[i:i+self.n_gram_len] for i in range(0, len(text) - self.n_gram_len + 1, 
                                                                  self.n_gram_len)]
        decoded_frequency_dict = Counter(splitted_text)
        sort_indx = np.argsort(list(decoded_frequency_dict.values()))[::-1]
        decoded_alphabet_sorted = np.array(list(decoded_frequency_dict.keys()))[sort_indx]
        transtable_size = min(len(decoded_alphabet_sorted), len(self._alphabet_sorted))
        transtable = {}
        for i in range(transtable_size):
            transtable[decoded_alphabet_sorted[i]] = self._alphabet_sorted[i]
        return ''.join([transtable[n_gram] for n_gram in splitted_text])[:original_len]

In [8]:
# Функция для расчета точности
def calc_accuracy(original_text, decoded_text):
    alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
    original_text = np.array([char for char in original_text.lower() if char in alphabet])
    decoded_text  = np.array(list(decoded_text))
    return (original_text == decoded_text).sum() / len(original_text)

In [9]:
# Создаем и обучаем частотный декодер на униграммах
decoder_unigram = Decoder(n_gram_len = 1)
decoder_unigram.fit(train_text)

In [10]:
# Посмотрим на частоты символов
sorted(decoder_unigram._frequency_dict.items(), key=lambda i: i[1], reverse = True)

[(' ', 286559),
 ('о', 162409),
 ('е', 123650),
 ('а', 117104),
 ('н', 98139),
 ('и', 93874),
 ('т', 84639),
 ('с', 75124),
 ('л', 70914),
 ('в', 66562),
 ('р', 56289),
 ('к', 48460),
 ('д', 41632),
 ('м', 40527),
 ('у', 38129),
 ('п', 34093),
 ('я', 30444),
 ('ь', 27853),
 ('ы', 26214),
 ('г', 25693),
 ('б', 24718),
 ('ч', 23858),
 ('з', 23121),
 ('ж', 16020),
 ('й', 14862),
 ('ш', 12068),
 ('х', 10986),
 ('ю', 8812),
 ('э', 5019),
 ('щ', 4054),
 ('ц', 3993),
 ('ф', 1781),
 ('ъ', 412),
 ('ё', 31)]

In [11]:
# Кодируем тестовый текст
encoded_text = encode(test_text)

In [12]:
print('Закодированный тестовый текст: ')
print(encoded_text)

Закодированный тестовый текст: 
ейншюяжфбящыраефбцфуышцдефяфбцфуывцшфйыэбкфыбефзюцаецшфбенфюынняжфыабефаыдибефокшмфнзеняшцдмбяёцэфцйюызкфбеьфодерыацшцдмфсбецшфнйыцфйкныщыцфзюясйебяцфяфо ацшфйцюцбфцл фйышфыабыфйыфвшыфжфйцютфбеьцл фаыоюыл фяфв абыл фрын аеютфзюцаншыяшфйцдявеэьежфюыдмфйфляюцфяфыбфшещфаыоюыацшцдцбфяфуыюыьфвшыфоырфбцфыншейяшфцры


In [13]:
# Декодируем закодированный текст униграммным декодером
decoded_text = decoder_unigram.decode(encoded_text)

In [14]:
print('Декодированный текст: ')
print(decoded_text)

Декодированный текст: 
нвристы атжоялн ае зоиекн т ае зоуеи войач оан ьселнеи анр соррты олан локюан мчиг рьнртиекгатэей евсоьч анб мкняолеиекг шанеи рвое вчрожое ьстшвнате т мдлеи весеа епд вои олао во уио ы весх анбепд ломсопд т удлаопд яордлнсх ьселриоти вектунйбны сокг в птсе т оа инж ломсолеиекеа т зособ уио моя ае оринвти еяо


In [15]:
print('Точность декодирования униграммным методом: ', calc_accuracy(test_text, decoded_text))

Точность декодирования униграммным методом:  0.42258064516129035


**Вывод:** при использовании базового частотного метода получается не очень высокая точность, расшифрованный текст сложно понять.

[К содержанию](#zero-bullet)

## 2. Частотный метод на базе биграмм <a class="anchor" id="2"></a>  

Условие задачи: *Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:*  
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;  
- проведите тестирование аналогично п.1, но при помощи биграмм

In [16]:
# Создаем и обучаем частотный декодер на биграммах
decoder_bigram = Decoder(n_gram_len = 2)
decoder_bigram.fit(train_text)

In [17]:
print('Количество биграмм в словаре:', len(decoder_bigram._frequency_dict))

Количество биграмм в словаре: 855


In [18]:
# Посмотрим на частоты биграмм
sorted(decoder_bigram._frequency_dict.items(), key=lambda i: i[1], reverse = True)

[('о ', 39425),
 ('е ', 31310),
 ('а ', 31156),
 ('и ', 30321),
 (' н', 27158),
 (' с', 26942),
 (' в', 24561),
 ('то', 24162),
 (' п', 23483),
 (' о', 22353),
 ('я ', 18580),
 (' и', 18511),
 ('на', 18425),
 ('ст', 17654),
 ('ь ', 17330),
 ('не', 16582),
 ('но', 16334),
 ('ал', 16156),
 (' к', 15001),
 ('го', 14059),
 ('он', 14029),
 ('по', 13701),
 (' т', 13160),
 ('ко', 13143),
 ('ни', 13006),
 ('ка', 12955),
 ('ла', 12779),
 ('ов', 12347),
 ('ен', 12293),
 (' д', 12152),
 ('л ', 12026),
 ('й ', 11983),
 ('во', 11773),
 ('ро', 11586),
 ('м ', 11566),
 ('ра', 11540),
 (' б', 11219),
 ('от', 10958),
 ('у ', 10852),
 (' ч', 10518),
 ('ть', 10409),
 ('пр', 10400),
 ('ос', 9889),
 ('ло', 9720),
 ('ел', 9661),
 (' м', 9284),
 ('ол', 9229),
 ('ва', 9180),
 ('ор', 9135),
 ('в ', 9105),
 ('н ', 9070),
 ('за', 8940),
 ('ли', 8852),
 ('ре', 8822),
 ('ле', 8791),
 (' е', 8770),
 ('ер', 8767),
 ('ск', 8478),
 ('те', 8393),
 ('та', 8326),
 ('  ', 8291),
 ('ом', 8114),
 ('ан', 7815),
 ('к ', 7803)

In [19]:
# Декодируем закодированный текст биграммным декодером
decoded_text = decoder_bigram.decode(encoded_text)

In [20]:
print('Декодированный текст: ')
print(decoded_text)

Декодированный текст: 
у гов а нокотьнеал чреронео ал ч мто сливам ст б вовто иелоспрлаа нист толорнен заонлолаь  пно евеоде ак ипоиты  оя ь  пазсттоонкаи быдакои ин зженои о емя ен д в кй  нкаенниес снавонаа  дот ипой  н ртира но  ут ра н ооговот б вчтскл  сротета  а е  п сомани о ерк ет твие я ь де ко иле ривонасеаталм гоу л ле о


In [21]:
print('Точность декодирования биграммным методом: ', calc_accuracy(test_text, decoded_text))

Точность декодирования биграммным методом:  0.14838709677419354


**Вывод:** при использовании биграммного метода точность получается еще ниже, чем в базовом методе.  

[К содержанию](#zero-bullet)

## 3. MCMC-метод <a class="anchor" id="3"></a>  

Условие задачи: *Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:*  
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;  
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.  

Закодированный текст получен из исходного с помощью некоторой перестановки символов. Необходимо найти такую обратную перестановку inv_perm, которая после применения к закодированному тексту даст наиболее вероятный с точки зрения распределения биграмм результат:  

$$P(\text{text}|\text{inv_perm}) \xrightarrow[\text{inv_perm}]{} \max$$

Для аппроксимации правдоподобия $P(\text{text}|\text{inv_perm})$ будем использовать алгоритм Метрополиса-Гастингса. При этом семплер будет менять местами произвольную пару символов, которые выбираются равновероятно. Для принятия или отвержения предложенной перестановки будем анализировать отношение правдоподобий:  
$$a = \frac{P(\text{text}|\text{inv_perm}_{\text{proposed}})}{P(\text{text}|\text{inv_perm}_{\text{current}})}$$  
Если $a \geq 1, $ то принимаем предложенную перестановку, иначе принимаем её с вероятностью $a.$

In [40]:
# Класс для декодирования по методу МСМС
class MCMCDecoder:
    
    def __init__(self):
        self._alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
        self._char_to_idx = {char: idx for idx, char in enumerate(self._alphabet)}
        self._transition_matrix = np.zeros((len(self._alphabet), len(self._alphabet)))
    
    def fit(self, text):
        text = ''.join([char for char in text.lower() if char in self._alphabet])
        for i in range(len(text) - 1):
            self._transition_matrix[self._char_to_idx[text[i]], self._char_to_idx[text[i+1]]] += 1
        self._transition_matrix = np.clip(self._transition_matrix, 1, None)
        self._transition_matrix = (np.log(self._transition_matrix).T 
                                   - np.log(self._transition_matrix.sum(axis=1))).T
        
    def _translate(self, text, permutation):
        transtable = str.maketrans(self._alphabet, ''.join(permutation))
        return text.translate(transtable)
        
    def _evaluate_log_likelihood(self, text, permutation):
        text = self._translate(text, permutation)
        log_likelihood = 0
        for i in range(len(text) - 1):
            log_likelihood += self._transition_matrix[self._char_to_idx[text[i]], self._char_to_idx[text[i+1]]]
        return log_likelihood
    
    def decode(self, text, n_iters = 10000, restart_period = None):
        permutation = np.array(list(self._alphabet))
        random.shuffle(permutation)
        current_log_likelihood = self._evaluate_log_likelihood(text, permutation)
        best_log_likelihood = current_log_likelihood
        best_permutation = permutation.copy()
        for i in range(n_iters):
            if restart_period is not None and i % restart_period == 0:
                random.shuffle(permutation)
                current_log_likelihood = self._evaluate_log_likelihood(text, permutation)
            swap_idx = random.sample(range(len(self._alphabet)), 2)
            permutation[swap_idx[0]], permutation[swap_idx[1]] = \
                permutation[swap_idx[1]], permutation[swap_idx[0]]
            new_log_likelihood = self._evaluate_log_likelihood(text, permutation)
            if new_log_likelihood >= current_log_likelihood:
                current_log_likelihood = new_log_likelihood
                if new_log_likelihood > best_log_likelihood:
                    best_log_likelihood = new_log_likelihood
                    best_permutation = permutation.copy()
            else:
                if random.random() < np.exp(new_log_likelihood - current_log_likelihood):
                    current_log_likelihood = new_log_likelihood
                else:
                    permutation[swap_idx[0]], permutation[swap_idx[1]] = \
                        permutation[swap_idx[1]], permutation[swap_idx[0]]
        return self._translate(text, best_permutation)

In [43]:
# Создаем МСМС декодер и обучаем его
mcmc_decoder = MCMCDecoder()
mcmc_decoder.fit(train_text)

In [44]:
# Декодируем текст с помощью МСМС
decoded_text = mcmc_decoder.decode(encoded_text, n_iters = 20000)
print('Декодированный текст: ')
print(decoded_text)
print()
print('Точность декодирования МСМС-методом: ', calc_accuracy(test_text, decoded_text))

Декодированный текст: 
австрия нихогда не котела и не кочет войны она предает нас россия одна должна быть спасительницей европы наш благодетель знает свое высохое призвание и будет верен ему вот одно во что я верю нашему доброму и чудному государю предстоит величайшая роль в мире и он тах добродетелен и корош что бог не оставит его

Точность декодирования МСМС-методом:  0.9806451612903225


**Выводы:**  
- при использовании МСМС-метода точность получается выше, чем в частотных методах;  
- поскольку алгоритм носит стохастический характер, иногда может потребоваться несколько запусков для того, чтобы сойтись к хорошему результату.  

[К содержанию](#zero-bullet)

## 4. Расшифровка сообщения <a class="anchor" id="4"></a>  

Условие задачи: *расшифруйте сообщение:*  
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏  

Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):  
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ  

In [45]:
message = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [46]:
print('Зашифрованное сообщение в исходном виде:')
print(message)

Зашифрованное сообщение в исходном виде:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏


Для начала приведем зашифрованное сообщение к русскому алфавиту

In [47]:
encoded_letters = ''.join(set(message))
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '[:len(encoded_letters)]
transtable = str.maketrans(encoded_letters, alphabet)
message = message.translate(transtable)
print('Зашифрованное сообщение в русском алфавите:')
print(message)

Зашифрованное сообщение в русском алфавите:
бёгиречреищизбруъцахгфучсригирйъжзируъцахгфучсрзбпёзртрвзъкърёъъомбуинрпъзъцчсргбкпърйцъжизхзфрёпъцббреёбкъречреёбрёщбгхгирйцхеигфуърирйъгтжизбрахпёиахгфучсрохггрдхрйъёгбщуббржбзебцзъбрдхщхуибрптцёхрлъзнрпъубжуърнруижбкърубръобмхш


In [49]:
# Декодируем текст с помощью частотного метода
message_decoded_baseline = decoder_unigram.decode(encoded_text)
print('Декодированный текст (частотный метод): ')
print(message_decoded_baseline)

Декодированный текст (частотный метод): 
нвристы атжоялн ае зоиекн т ае зоуеи войач оан ьселнеи анр соррты олан локюан мчиг рьнртиекгатэей евсоьч анб мкняолеиекг шанеи рвое вчрожое ьстшвнате т мдлеи весеа епд вои олао во уио ы весх анбепд ломсопд т удлаопд яордлнсх ьселриоти вектунйбны сокг в птсе т оа инж ломсолеиекеа т зособ уио моя ае оринвти еяо


In [50]:
# Декодируем текст с помощью МСМС
message_decoded = mcmc_decoder.decode(message, n_iters=500000, restart_period=200000)

In [51]:
print('Декодированный текст (МСМС-метод): ')
print(message_decoded)

Декодированный текст (МСМС-метод): 
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж


**Ответ:** если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж

**Выводы:**  
- использовании МСМС-метода позволило прочитать зашифрованное сообщение;  
- для получения кондиционного результата понадобилось выставить большое количество итераций;  
- понадобилось несколько запусков алгоритма декодирования.   

[К содержанию](#zero-bullet)

## 5. Бонус 1 <a class="anchor" id="5"></a>  
Условие задачи: *бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.*

## 6. Бонус 2 <a class="anchor" id="6"></a>  
Условие задачи: *бонус: какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).*  

**Возможные применения МСМС-метода**:  
- восстановление исходного текстового сообщения при нарушении или сбое кодировки;  
- решение задачи о рюкзаке: укладка как можно большего числа нужных вещей в рюкзак при условии, что общий объём (или вес) всех предметов, способных поместиться в рюкзак, ограничен.  

[К содержанию](#zero-bullet)